# Assignment 5: Visual Servoing

#### for every question this code need a restart to reset the environment.

The following has ALL the code to run the simulation and all the problems. You do not need to understand exactly how it works. You only need to work on the sections starting at "COMPLETE THIS". 

In [ ]:
#import matplotlib.pyplot as plt
import numpy as np
import pybullet as p
import pybullet_data
import os
import time
import cv2
from scipy.spatial.transform import Rotation as Rot            #can use this to apply angular rotations to coordinate frames

#camera (don't change these settings)
camera_width = 512                                             #image width
camera_height = 512                                            #image height
camera_fov = 120                                                #field of view of camera
camera_focal_depth = 0.5*camera_height/np.tan(0.5*np.pi/180*camera_fov) 
                                                               #focal depth in pixel space
camera_aspect = camera_width/camera_height                     #aspect ratio
camera_near = 0.02                                             #near clipping plane in meters, do not set non-zero
camera_far = 100                                               #far clipping plane in meters


#control objectives (if you wish, you can play with these values for fun)
object_location_desired = np.array([camera_width/2,camera_height/2])

                                                               #center the object to middle of image 
K_p_x = 0.1                                                   #Proportional control gain for translation
K_p_Omega = 0.02                                              #Proportional control gain for rotation       

### Create the Robot Class

In [ ]:
# Robot with Camera Class
class eye_in_hand_robot:
    def get_ee_position(self):
        '''
        Function to return the end-effector of the link. This is the very tip of the robot at the end of the jaws.
        '''
        endEffectorIndex = self.numActiveJoints
        endEffectorState = p.getLinkState(self.robot_id, endEffectorIndex)
        endEffectorPos = np.array(endEffectorState[0])
        endEffectorOrn = np.array(p.getMatrixFromQuaternion(endEffectorState[1])).reshape(3,3)
        
        #add an offset to get past the forceps
        endEffectorPos += self.camera_offset*endEffectorOrn[:,2]
        return endEffectorPos, endEffectorOrn

    def get_current_joint_angles(self):
        # Get the current joint angles
        joint_angles = np.zeros(self.numActiveJoints)
        for i in range(self.numActiveJoints):
            joint_state = p.getJointState(self.robot_id, self._active_joint_indices[i])
            joint_angles[i] = joint_state[0]
        return joint_angles
    
    def get_jacobian_at_current_position(self):
        #Returns the Robot Jacobian of the last active link
        mpos, mvel, mtorq = self.get_active_joint_states()   
        zero_vec = [0.0]*len(mpos)
        linearJacobian, angularJacobian = p.calculateJacobian(self.robot_id, 
                                                              self.numActiveJoints,
                                                              [0,0,self.camera_offset],
                                                              mpos, 
                                                              zero_vec,
                                                              zero_vec)
        #only return the active joint's jacobians
        Jacobian = np.vstack((linearJacobian,angularJacobian))
        return Jacobian[:,:self.numActiveJoints]
    
    def set_joint_position(self, desireJointPositions, kp=1.0, kv=0.3):
        '''Set  the joint angle positions of the robot'''
        zero_vec = [0.0] * self._numLinkJoints
        allJointPositionObjectives = [0.0]*self._numLinkJoints
        for i in range(desireJointPositions.shape[0]):
            idx = self._active_joint_indices[i]
            allJointPositionObjectives[idx] = desireJointPositions[i]

        p.setJointMotorControlArray(self.robot_id,
                                    range(self._numLinkJoints),
                                    p.POSITION_CONTROL,
                                    targetPositions=allJointPositionObjectives,
                                    targetVelocities=zero_vec,
                                    positionGains=[kp] * self._numLinkJoints,
                                    velocityGains=[kv] * self._numLinkJoints)

    def get_active_joint_states(self):
        '''Get the states (position, velocity, and torques) of the active joints of the robot
        '''
        joint_states = p.getJointStates(self.robot_id, range(self._numLinkJoints))
        joint_infos = [p.getJointInfo(self.robot_id, i) for i in range(self._numLinkJoints)]
        joint_states = [j for j, i in zip(joint_states, joint_infos) if i[3] > -1]
        joint_positions = [state[0] for state in joint_states]
        joint_velocities = [state[1] for state in joint_states]
        joint_torques = [state[3] for state in joint_states]
        return joint_positions, joint_velocities, joint_torques


         
    def __init__(self, robot_id, initialJointPos):
        self.robot_id = robot_id
        self.eeFrameId = []
        self.camera_offset = 0.1 #offset camera in z direction to avoid grippers
        # Get the joint info
        self._numLinkJoints = p.getNumJoints(self.robot_id) #includes passive joint
        jointInfo = [p.getJointInfo(self.robot_id, i) for i in range(self._numLinkJoints)]
        
        # Get joint locations (some joints are passive)
        self._active_joint_indices = []
        for i in range(self._numLinkJoints):
            if jointInfo[i][2]==p.JOINT_REVOLUTE:
                self._active_joint_indices.append(jointInfo[i][0])
        self.numActiveJoints = len(self._active_joint_indices) #exact number of active joints

        #reset joints
        for i in range(self._numLinkJoints):
            p.resetJointState(self.robot_id,i,initialJointPos[i])

### Ancilliary Functions

In [ ]:
def draw_coordinate_frame(position, orientation, length, frameId = []):
    '''
    Draws a coordinate frame x,y,z with scaled lengths on the axes 
    in a position and orientation relative to the world coordinate frame
    pos: 3-element numpy array
    orientation: 3x3 numpy matrix
    length: length of the plotted x,y,z axes
    frameId: a unique ID for the frame. If this supplied, then it will erase the previous location of the frame
    
    returns the frameId
    '''
    if len(frameId)!=0:
        p.removeUserDebugItem(frameId[0])
        p.removeUserDebugItem(frameId[1])
        p.removeUserDebugItem(frameId[2])
    
    lineIdx=p.addUserDebugLine(position, position + np.dot(orientation, [length, 0, 0]), [1, 0, 0])  # x-axis in red
    lineIdy=p.addUserDebugLine(position, position + np.dot(orientation, [0, length, 0]), [0, 1, 0])  # y-axis in green
    lineIdz=p.addUserDebugLine(position, position + np.dot(orientation, [0, 0, length]), [0, 0, 1])  # z-axis in blue

    return lineIdx,lineIdy,lineIdz

def opengl_plot_world_to_pixelspace(pt_in_3D_to_project, viewMat, projMat, imgWidth, imgHeight):
    ''' Plots a x,y,z location in the world in an openCV image
    This is used for debugging, e.g. given a known location in the world, verify it appears in the camera
    when using p.getCameraImage(...). The output [u,v], when plot with opencv, should line up with object 
    in the image from p.getCameraImage(...)
    '''
    pt_in_3D_to_project = np.append(pt_in_3D_to_project,1)
    #print('Point in 3D to project:', pt_in_3D_to_project)

    pt_in_3D_in_camera_frame = viewMat @ pt_in_3D_to_project
    #print('Point in camera space: ', pt_in_3D_in_camera_frame)

    # Convert coordinates to get normalized device coordinates (before rescale)
    uvzw = projMat @ pt_in_3D_in_camera_frame
    #print('after projection: ', uvzw)

    # scale to get the normalized device coordinates
    uvzw_NDC = uvzw/uvzw[3]
    #print('after normalization: ', uvzw_NDC)

    #x,y specifies lower left corner of viewport rectangle, in pixels. initial value is (0,)
    u = ((uvzw_NDC[0] + 1) / 2.0) * imgWidth
    v = ((1-uvzw_NDC[1]) / 2.0) * imgHeight

    return [int(u),int(v)]

    
def get_camera_view_and_projection_opencv(cameraPos, camereaOrn):
    '''Gets the view and projection matrix for a camera at position (3) and orientation (3x3)'''
    __camera_view_matrix_opengl = p.computeViewMatrix(cameraEyePosition=cameraPos,
                                                   cameraTargetPosition=cameraPos+camereaOrn[:,2],
                                                   cameraUpVector=-camereaOrn[:,1])

    __camera_projection_matrix_opengl = p.computeProjectionMatrixFOV(camera_fov, camera_aspect, camera_near, camera_far)        
    _, _, rgbImg, depthImg, _ = p.getCameraImage(camera_width, 
                                                 camera_height, 
                                                 __camera_view_matrix_opengl,
                                                 __camera_projection_matrix_opengl, 
                                                 renderer=p.ER_BULLET_HARDWARE_OPENGL)

    #returns camera view and projection matrices in a form that fits openCV
    viewMat = np.array(__camera_view_matrix_opengl).reshape(4,4).T
    projMat = np.array(__camera_projection_matrix_opengl).reshape(4,4).T
    return viewMat, projMat

def get_camera_img_float(cameraPos, camereaOrn):
    ''' Gets the image and depth map from a camera at a position cameraPos (3) and cameraOrn (3x3) in space. '''
    __camera_view_matrix_opengl = p.computeViewMatrix(cameraEyePosition=cameraPos,
                                                   cameraTargetPosition=cameraPos+camereaOrn[:,2],
                                                   cameraUpVector=-camereaOrn[:,1])

    __camera_projection_matrix_opengl = p.computeProjectionMatrixFOV(camera_fov, camera_aspect, camera_near, camera_far)        
    width, height, rgbImg, nonlinDepthImg, _ = p.getCameraImage(camera_width, 
                                                 camera_height, 
                                                 __camera_view_matrix_opengl,
                                                 __camera_projection_matrix_opengl, 
                                                 renderer=p.ER_BULLET_HARDWARE_OPENGL)

    #adjust for clipping and nonlinear distance i.e., 1/d (0 is closest, i.e., near, 1 is furthest away, i.e., far
    depthImgLinearized =camera_far*camera_near/(camera_far+camera_near-(camera_far-camera_near)*nonlinDepthImg)

    #convert to numpy and a rgb-d image
    rgb_image = np.array(rgbImg[:,:,:3], dtype=np.uint8)
    depth_image = np.array(depthImgLinearized, dtype=np.float32)
    return rgb_image, depth_image


### Create Physics Environment

In [ ]:
# Start the connection to the physics server
physicsClient = p.connect(p.GUI)#(p.DIRECT)
time_step = 0.001
p.resetSimulation()
p.setTimeStep(time_step)
p.setGravity(0, 0, -9.8)

# Set the path to the URDF files included with PyBullet
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# load a plane URDF
p.loadURDF('plane.urdf')

# Place obstacles
box_length = box_width = 0.4
box_depth = 0.02
object_center = [0.3, 1, 0.01]
object_orientation = [0, 0, 0]
object_color = [0.8, 0.0, 0.0, 1]
geomBox = p.createCollisionShape(p.GEOM_BOX, halfExtents=[box_length/2, box_width/2, box_depth/2])
visualBox = p.createVisualShape(p.GEOM_BOX, halfExtents=[box_length/2, box_width/2, box_depth/2], rgbaColor=object_color)
boxId = p.createMultiBody(
    baseMass=0,
    baseCollisionShapeIndex=geomBox,
    baseVisualShapeIndex=visualBox,
    basePosition=np.array(object_center),
    baseOrientation=p.getQuaternionFromEuler(object_orientation)
)
ObjectModelPos, modelOrn  = p.getBasePositionAndOrientation(boxId)

#reset debug gui camera position so we can see the robot up close
p.resetDebugVisualizerCamera( cameraDistance=1, cameraYaw=30, cameraPitch=-52, cameraTargetPosition=[0,0,.5])


### COMPLETE THIS

In [ ]:
def getImageJacobian(u_px, v_px, depthImg, f, imgWidth, imgHeight, cameraOrientation):
    u_idx = int(np.clip(round(u_px), 0, depthImg.shape[1] - 1))
    v_idx = int(np.clip(round(v_px), 0, depthImg.shape[0] - 1))
    Z = float(depthImg[v_idx, u_idx])
    if Z < 0.01:
        Z = 0.01

    print("u_idx, v_idx, Z", u_idx, v_idx, Z)
    x = (u_px - imgWidth/2.0) / f
    y = (v_px - imgHeight/2.0) / f

    L_norm = np.array([
        [-1.0/Z,  0.0,    x/Z,          x*y,          -(1.0 + x*x),   y],
        [ 0.0,   -1.0/Z,  y/Z,          1.0 + y*y,    -x*y,          -x]
    ], dtype=float)

    L_cam = np.empty_like(L_norm)
    L_cam[0,:] =  f * L_norm[0,:]
    L_cam[1,:] =  f * L_norm[1,:]

    # camera to world   
    R_wc = np.asarray(cameraOrientation, dtype=float)
    R_cw = R_wc.T
    T = np.block([
        [R_cw,            np.zeros((3,3))],
        [np.zeros((3,3)), R_cw]
    ])

    L_world = L_cam @ T

    return L_world

#### comment there to solve only translation or rotation in Q2

In [ ]:
def findCameraControl(object_loc_des, object_loc, image_jacobian):    
    error = np.array(object_loc_des) - np.array(object_loc)
    
    # Use only translation
    # image_jacobian[:, 3:6] = 0
    
    # Use only rotation
    # image_jacobian[:, 0:3] = 0
        
    J_pseudo_inv = np.linalg.pinv(image_jacobian) 
    
    camera_velocity = J_pseudo_inv @ error
    
    velocity_trans = camera_velocity[0:3]  
    velocity_rot = camera_velocity[3:6]   
    
    delta_X = K_p_x * velocity_trans          
    delta_Omega = K_p_Omega * velocity_rot    
    return delta_X, delta_Omega

In [ ]:
def findJointControl(robot, delta_X, delta_Omega):
    current_joint_angles = robot.get_current_joint_angles()
    
    J_robot = robot.get_jacobian_at_current_position()
    
    camera_velocity = np.hstack((delta_X, delta_Omega))
    
    joint_velocity = np.linalg.pinv(J_robot) @ camera_velocity
    
    new_jointPositions = current_joint_angles + joint_velocity
    
    return new_jointPositions

## SIMULATE ANSWERS

### Q1 and Q2 Setup

In [ ]:
cam_pos_init = np.array([0.2, 0.0, 1.0])
cam_rot_init = np.array([[1,0,0],[0,-1,0],[0,0,-1]])
axes = [
    ("+vx", 0),
    ("+vy", 1),
    ("+vz", 2),
    ("+wx", 3),
    ("+wy", 4),
    ("+wz", 5),
]

camera_frameId = []

for name, id in axes:
    print(f"\n=== Axis: {name} ===")
    log_id = p.startStateLogging(p.STATE_LOGGING_VIDEO_MP4, f"Q1_{name}.mp4")
    cameraPosition    = cam_pos_init.copy()
    cameraOrientation = cam_rot_init.copy()

    for t in range(20):
        p.stepSimulation()

        rgb, depth = get_camera_img_float(cameraPosition, cameraOrientation)
        viewMat, projMat = get_camera_view_and_projection_opencv(cameraPosition, cameraOrientation)
        object_loc = opengl_plot_world_to_pixelspace(object_center, viewMat, projMat, camera_width, camera_height)

        L = getImageJacobian(object_loc[0], object_loc[1], depth, camera_focal_depth,
                             camera_width, camera_height, cameraOrientation)
        delta_X, delta_Omega = findCameraControl(object_location_desired, object_loc, L)
        control = np.concatenate((delta_X, delta_Omega), axis=0)
        L_inv = np.zeros((6,))

        # enlarger the control
        L_inv[id] = control[id] * 10
        delta_X = L_inv[0:3]
        delta_Omega = L_inv[3:6]

        cameraPosition = cameraPosition + delta_X

        omega_mag = np.linalg.norm(delta_Omega)
        if omega_mag > 1e-9:
            rotation_update   = Rot.from_rotvec(delta_Omega)
            current_rotation  = Rot.from_matrix(cameraOrientation)
            cameraOrientation = (rotation_update * current_rotation).as_matrix()

        if t % 1 == 0:
            camera_frameId = draw_coordinate_frame(cameraPosition, cameraOrientation, 
                                                length=0.15, frameId=camera_frameId)
        
        if t % 1 == 0:
            error = np.array(object_location_desired) - np.array(object_loc)
            print(f"Iter {t}: Error={np.linalg.norm(error):.2f}px, Obj@{object_loc}, Des@{object_location_desired}")

        # cv2.imshow("depth", depth)
        # cv2.imshow("rgb", rgb)
        # cv2.waitKey(1)
        
    p.stopStateLogging(log_id)
cv2.destroyAllWindows()
p.disconnect()


In [ ]:
cameraPosition = np.array([0.2,0,1])
cameraOrientation = np.array([[1,0,0],[0,-1,0],[0,0,-1]])

camera_frameId = []
log_id = p.startStateLogging(p.STATE_LOGGING_VIDEO_MP4, "test.mp4")

for ITER in range(100):
    p.stepSimulation()
 
    ''' Get Image'''
    rgb, depth = get_camera_img_float(cameraPosition, cameraOrientation)
    
    ''' Magical Computer Vision Algorithm that gets locations of objects in the image, as object_loc (Do Not Remove)'''
    viewMat, projMat = get_camera_view_and_projection_opencv(cameraPosition, cameraOrientation)
    object_loc = opengl_plot_world_to_pixelspace(object_center, viewMat, projMat,camera_width, camera_height)
    
    ''' Get Image Jacobian '''
    imageJacobian = getImageJacobian(object_loc[0], object_loc[1], depth, camera_focal_depth, camera_width, camera_height, cameraOrientation)

    ''' Get Camera Control '''
    delta_X, delta_Omega = findCameraControl(object_location_desired, object_loc, imageJacobian)

    cameraPosition = cameraPosition + delta_X

    omega_mag = np.linalg.norm(delta_Omega)
    if omega_mag > 1e-6:
        rotation_update   = Rot.from_rotvec(delta_Omega)  # 世界系旋量
        current_rotation  = Rot.from_matrix(cameraOrientation)
        cameraOrientation = (rotation_update * current_rotation).as_matrix()
    
    if ITER % 1 == 0:
        camera_frameId = draw_coordinate_frame(cameraPosition, cameraOrientation, 
                                               length=0.15, frameId=camera_frameId)
    
    if ITER % 1 == 0:
        error = np.array(object_location_desired) - np.array(object_loc)
        print(f"Iter {ITER}: Error={np.linalg.norm(error):.2f}px, Obj@{object_loc}, Des@{object_location_desired}")

    # # show image
    # cv2.imshow("depth", depth)
    # cv2.imshow("rgb", rgb)
    # cv2.waitKey(1)
    
p.stopStateLogging(log_id)

#close the physics server
cv2.destroyAllWindows()    
p.disconnect() 

### Q3_a(object not moving)

In [ ]:
''' Create Robot Instance'''
pandaUid = p.loadURDF(os.path.join(pybullet_data.getDataPath(), "franka_panda/panda.urdf"),useFixedBase=True)
p.resetBasePositionAndOrientation(pandaUid, [0, 0, 0], [0, 0, 0, 1])
initialJointPosition = [0,-np.pi/4,np.pi/4,-np.pi/4,np.pi/4,np.pi/4,np.pi/4,0,0,0,0,0]
robot = eye_in_hand_robot(pandaUid,initialJointPosition)
p.stepSimulation() # need to do this to initialize robot

log_id = p.startStateLogging(p.STATE_LOGGING_VIDEO_MP4, "Q3_stationary.mp4")
camera_frameId = []

for ITER in range(200):
    p.stepSimulation()
 
    ''' Match Camera Pose to Robot End-Effector and Get Image'''
    cameraPosition, cameraOrientation = robot.get_ee_position()

    camera_frameId = draw_coordinate_frame(cameraPosition, cameraOrientation, 
                                        length=0.15, frameId=camera_frameId)
    
    rgb, depth = get_camera_img_float(cameraPosition, cameraOrientation)
    
    ''' Magical Computer Vision Algorithm that gets locations of objects in the image, as object_loc (Do Not Remove)'''
    viewMat, projMat = get_camera_view_and_projection_opencv(cameraPosition, cameraOrientation)
    object_loc = opengl_plot_world_to_pixelspace(object_center, viewMat, projMat,camera_width, camera_height)

    imageJacobian = getImageJacobian(object_loc[0], object_loc[1], depth, 
                                      camera_focal_depth, camera_width, camera_height, cameraOrientation)
    
    delta_X, delta_Omega = findCameraControl(object_location_desired, object_loc, imageJacobian)
    delta_X = delta_X * 0.1
    delta_Omega = delta_Omega * 0.1
    
    new_jointPositions = findJointControl(robot, delta_X, delta_Omega)
    
    robot.set_joint_position(new_jointPositions)
    
    if ITER % 10 == 0:
        error = np.linalg.norm(np.array(object_location_desired) - np.array(object_loc))
        print(f"Iter {ITER}: Error={error:.2f}px, Object位置={object_loc}, 目标位置={object_location_desired.tolist()}")
    
    cv2.imshow("depth", depth)
    cv2.imshow("rgb", rgb)
    cv2.waitKey(1)

p.stopStateLogging(log_id)

#close the physics server
cv2.destroyAllWindows()    
p.disconnect() 

### Q3_b(Track Moving Target)

In [ ]:
''' Create Robot Instance'''
pandaUid = p.loadURDF(os.path.join(pybullet_data.getDataPath(), "franka_panda/panda.urdf"),useFixedBase=True)
p.resetBasePositionAndOrientation(pandaUid, [0, 0, 0], [0, 0, 0, 1])
initialJointPosition = [0,-np.pi/4,np.pi/4,-np.pi/4,np.pi/4,np.pi/4,np.pi/4,0,0,0,0,0]
robot = eye_in_hand_robot(pandaUid,initialJointPosition)
p.stepSimulation() # need to do this to initialize robot

log_id = p.startStateLogging(p.STATE_LOGGING_VIDEO_MP4, "Q3_moving.mp4")
cameraframeId = []

for ITER in range(300):
    p.stepSimulation()
    
    object_center=[0.5+0.2*np.sin(np.pi/2+ITER/10), 0.5+0.2*np.sin(ITER/20), 0.01]
    p.resetBasePositionAndOrientation(boxId,object_center,p.getQuaternionFromEuler(object_orientation))
 
    ''' Match Camera Pose to Robot End-Effector and Get Image'''
    cameraPosition, cameraOrientation = robot.get_ee_position()
    
    rgb, depth = get_camera_img_float(cameraPosition, cameraOrientation)
    
    ''' Magical Computer Vision Algorithm that gets locations of objects in the image, as object_loc (Do Not Remove)'''
    viewMat, projMat = get_camera_view_and_projection_opencv(cameraPosition, cameraOrientation)
    object_loc = opengl_plot_world_to_pixelspace(object_center, viewMat, projMat,camera_width, camera_height)

    ''' Do some things here to get robot control'''
    imageJacobian = getImageJacobian(object_loc[0], object_loc[1], depth, 
                                      camera_focal_depth, camera_width, camera_height, cameraOrientation)
    
    delta_X, delta_Omega = findCameraControl(object_location_desired, object_loc, imageJacobian)
    delta_X = delta_X * 0.3
    delta_Omega = delta_Omega * 0.3
    
    new_jointPositions = findJointControl(robot, delta_X, delta_Omega)
    
    robot.set_joint_position(new_jointPositions)
    
    if ITER % 10 == 0:
        error = np.linalg.norm(np.array(object_location_desired) - np.array(object_loc))
        print(f"Iter {ITER}: Error={error:.2f}px, Object location={object_loc}, Target location={object_location_desired.tolist()}")
    
    cv2.imshow("depth", depth)
    cv2.imshow("rgb", rgb)
    cv2.waitKey(1)

# 停止录制
p.stopStateLogging(log_id)

#close the physics server
cv2.destroyAllWindows()    
p.disconnect()


### Q3_c(Track Moving Target with no roll)

In [ ]:
def compute_roll_twist_sec(cameraOrientation, k_roll=1.0):
    """
    Compute the secondary roll twist for the camera to maintain zero roll 
    with respect to the world z-axis.

    Args:
        cameraOrientation (np.ndarray): The 3x3 rotation matrix representing the camera orientation in world coordinates.
        k_roll (float, optional): Gain parameter for roll correction. Default is 1.0.

    Returns:
        psi (float): Roll angle (in radians) of the camera around the viewing axis (z) with respect to the world z-axis.
        v_sec_world (np.ndarray): 6D twist vector in the world frame, representing the desired secondary control to minimize roll.
                                 The first 3 elements are zeros (no translation), and the last 3 are the angular velocity.
    """
    R_wc = np.asarray(cameraOrientation, dtype=float)   
    xw, yw, zw = R_wc[:,0], R_wc[:,1], R_wc[:,2]      
    g  = np.array([0.0, 0.0, 1.0])                    
    g_perp = g - np.dot(g, zw) * zw                
    nrm = np.linalg.norm(g_perp)
    if nrm < 1e-9:
        return 0.0, np.zeros(6)                       
    g_perp /= nrm

    psi = np.arctan2(np.dot(xw, g_perp), np.dot(yw, g_perp))
    omega_sec = -k_roll * psi * zw                    
    v_sec_world = np.zeros(6)
    v_sec_world[3:] = omega_sec
    return psi, v_sec_world

In [ ]:
''' Create Robot Instance'''
pandaUid = p.loadURDF(os.path.join(pybullet_data.getDataPath(), "franka_panda/panda.urdf"),useFixedBase=True)
p.resetBasePositionAndOrientation(pandaUid, [0, 0, 0], [0, 0, 0, 1])
initialJointPosition = [0,-np.pi/4,np.pi/4,-np.pi/4,np.pi/4,np.pi/4,np.pi/4,0,0,0,0,0]
robot = eye_in_hand_robot(pandaUid,initialJointPosition)
p.stepSimulation() # need to do this to initialize robot

log_id = p.startStateLogging(p.STATE_LOGGING_VIDEO_MP4, "Q3_no_roll.mp4")
cameraframeId = []

for ITER in range(300):
    p.stepSimulation()
    
    object_center=[0.5+0.2*np.sin(np.pi/2+ITER/10), 0.5+0.2*np.sin(ITER/20), 0.01]
    p.resetBasePositionAndOrientation(boxId,object_center,p.getQuaternionFromEuler(object_orientation))
 
    ''' Match Camera Pose to Robot End-Effector and Get Image'''
    cameraPosition, cameraOrientation = robot.get_ee_position()
    
    rgb, depth = get_camera_img_float(cameraPosition, cameraOrientation)
    
    ''' Magical Computer Vision Algorithm that gets locations of objects in the image, as object_loc (Do Not Remove)'''
    viewMat, projMat = get_camera_view_and_projection_opencv(cameraPosition, cameraOrientation)
    object_loc = opengl_plot_world_to_pixelspace(object_center, viewMat, projMat,camera_width, camera_height)

    ''' Do some things here to get robot control'''
    L_world = getImageJacobian(object_loc[0], object_loc[1], depth,
                                    camera_focal_depth, camera_width, camera_height,
                                    cameraOrientation)              # 2x6

    delta_X, delta_Omega = findCameraControl(object_location_desired, object_loc, L_world)
    delta = np.concatenate((delta_X, delta_Omega))
    kr = 1.0
    psi, v_sec_raw_world = compute_roll_twist_sec(cameraOrientation, k_roll=kr)

    Lplus = np.linalg.pinv(L_world)
    N = np.eye(6) - Lplus @ L_world
    v_cmd_world = delta + N @ v_sec_raw_world                 # 6,

    v_cmd_world[:3]  *= 0.1
    v_cmd_world[3:]  *= 0.1  

    delta_X_world     = v_cmd_world[:3]
    delta_Omega_world = v_cmd_world[3:]
    new_jointPositions = findJointControl(robot, delta_X_world, delta_Omega_world)

    robot.set_joint_position(new_jointPositions)

    if ITER % 10 == 0:
        error = np.linalg.norm(np.array(object_location_desired) - np.array(object_loc))
        print(f"Iter {ITER}: Error={error:.2f}px, Object位置={object_loc}, 目标位置={object_location_desired.tolist()}")
    
    cv2.imshow("depth", depth)
    cv2.imshow("rgb", rgb)
    cv2.waitKey(1)

# 停止录制
p.stopStateLogging(log_id)

#close the physics server
cv2.destroyAllWindows()    
p.disconnect()
